In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
os.chdir('/home/adam/adam/causal_inference')

from causal_inference.make_data.make_data import UseCaseLoader

from causal_inference.make_data.make_artificial_sessions import *
from causal_inference.make_data.make_proning_sessions import *
from causal_inference.make_data.make_covariates import *

In [ ]:
dl = UseCaseLoader()

In [ ]:
raw_data_path = '/home/adam/adam/data/causal_inference/data/raw/position_measurements.csv'
data_sessions_path = '/home/adam/adam/data/causal_inference/data/interim/unique_sessions_test.csv'
data_artificial_sessions = '/home/adam/adam/data/causal_inference/data/interim/artificial_sessions_test.csv'
data_inclusion_path = '/home/adam/adam/data/causal_inference/data/interim/incusion_sessions_test.csv'
data_covariates_path = '/home/adam/adam/data/causal_inference/data/interim/covariates.csv'
data_outcome_path = '/home/adam/adam/data/causal_inference/data/interim/outcomes.csv'
data_patients_path = '/home/adam/adam/data/causal_inference/data/interim/patients.csv'
data_medications_path = '/home/adam/adam/data/causal_inference/data/interim/medications.csv'
data_included_path = '/home/adam/adam/data/causal_inference/data/interim/included.csv'

In [ ]:
dl.get_position_measurements(save_path=raw_data_path)

In [ ]:
dl.make_unique_sessions(load_path=raw_data_path,
                        save_path=data_sessions_path)

In [ ]:
dl.add_artificial_sessions(load_path=data_sessions_path,
                           save_path=data_artificial_sessions)

In [ ]:
dl.add_inclusion_criteria(load_path=data_artificial_sessions,
                          save_path=data_inclusion_path)

In [ ]:
dl.apply_inclusion_criteria(load_path=data_inclusion_path,
                            save_path=data_included_path)

In [ ]:
dl.add_outcomes(load_path=data_included_path, save_path=data_outcome_path)

In [ ]:
dl.add_covariates(load_path=data_medications_path, save_path=data_covariates_path)

In [ ]:
dl.add_medications(load_path=data_outcome_path, save_path=data_medications_path)

In [ ]:
dl.add_patient_data(load_path=data_included_path, save_path=data_patients_path)


In [ ]:
df = load_position_data(data_covariates_path)

In [ ]:
df.loc[df.effective_value == 'prone', 'duration_hours'].describe(percentiles=[0.15, 0.25, 0.75, 0.9, 0.95])

In [ ]:
df.info()